<a href="https://colab.research.google.com/github/IA2122/practica-9-paochoa/blob/main/practica9_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 9 Parte 1: Word embeddings

En este primer notebook veremos distintos word embeddings. Para ello usaremos una librería de procesamiento de lenguaje natural llamada [gensim](https://radimrehurek.com/gensim/index.html). 

## Modelo antiguo

Antes de hablar de los modelos modernos, conviene conocer los modelos antiguos y sus limitaciones. 

Una de las técnicas más utilizadas hasta hace poco para codificar frases era conocida como [bolsa de palabras (o *bag of words*)](https://en.wikipedia.org/wiki/Bag-of-words_model). Esta técnica transforma cada documento a un vector de enteros de longitud fija. 

Por ejemplo, para las siguientes dos frases:

1. A Juan le gusta ver películas. A María también le gusta.
2. A Juan le gusta ver partidos de fútbol. María odia el futbol. 

El modelo produce los siguientes vectores (cada fila de la siguiente tabla representa una de las frases).


| Palabras | A | Juan | le | gusta | ver | películas | María | también | partidos | fútbol | odia | el |
|---------|--------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- | --------- |
|Frase 1|2 | 1 | 2 | 1 | 1 | 1 | 1 |1 | 0 | 0 | 0 | 0 |
|Frase 2|1 | 1 | 1 | 1 | 1 | 0 | 1 |0 | 1 | 1 | 1 | 1 |

Cada vector tiene 12 elementos, donde cada elemento cuenta el número de veces que una determinada palabra ocurre en dicho documento. 

Este modelo era muy efectivo, pero tenía varias limitaciones. En primer lugar se pierde información sobre el orden de las palabras. Por ejemplo, las frases "A María le gusta Juan" y "A Juan le gusta María" tienen vectores idénticos. La solución consiste en usar [n-gramas](https://en.wikipedia.org/wiki/N-gram). Además este modelo tiene la limitación de ser *sparse* (muchos ceros para cada vector) y tener una dimensionalidad muy alta. 

Otro de los problemas más importantes es que este modelo no aprende el significado de las palabras subyacentes, y como consecuencia la distancia entre vectores no refleja la similitud o diferencia en significado. Estos problemas se han resuelto con los word-embeddings.





## Word2Vec

El modelo [Word2Vec](https://arxiv.org/pdf/1301.3781.pdf) fue introducido en 2013 por investigadores de Google, y es capaz de incrustar las palabras en un vector de dimensión bja usando una red neuronal. El resultado es un modelo donde los vectores que están cerca en el espacio tienen significados similares basados en el contexto. Es decir, no existe un único modelo word2vec, sino que es un modelo que se entrena a partir de un dataset (dicho dataset puede ser la wikipedia, twitter, ...).

Vamos a ver que es posible hacer con uno de estos modelos. Para ello vamos a usar un modelo entrenado en el dataset de noticias de Google. Por el momento vamos a usar modelos entrenados en inglés, más adelante en la práctica veremos cómo usar modelos en español. 

Comenzamos descargando el dataset (esto puede costar un tiempo). 

In [ ]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

Una tarea común es obtener las primeras palabras del vocabulario usado para construir el modelo. 

In [ ]:
for i, word in enumerate(wv.vocab):
    if i == 10:
        break
    print(word)

</s>
in
for
that
is
on
##
The
with
said


Ahora podemos ver cuál es la representación de una palabra. 

Su representación no es de las 4 letras que la componen sólo, si no de su contexto y todo lo que acarrea una palabra.

In [ ]:
wv['king']

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

Desafortunadamente el modelo no es capaz de inferir el vector asociado a palabras "raras". 

In [ ]:
try:
    vec_cameroon = wv['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

The word 'cameroon' does not appear in this model


Estos modelos proporcionan distintas funciones para tratar varios problemas. 

Por ejemplo, podemos ver la similaridad entre palabras. 


In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'car'	'minivan'	0.69
'car'	'bicycle'	0.54
'car'	'airplane'	0.42
'car'	'cereal'	0.14
'car'	'communism'	0.06


Podemos también mostrar las 5 palabras más similares a *car*.

In [ ]:
print(wv.most_similar(positive=['car'], topn=5))

[('vehicle', 0.7821096181869507), ('cars', 0.7423830032348633), ('SUV', 0.7160962820053101), ('minivan', 0.6907036304473877), ('truck', 0.6735789775848389)]


**Ejercicio**: Busca las 5 palabras más similares para Spain. 

In [ ]:
print(wv.most_similar(positive=['Spain'], topn=5))

[('Portugal', 0.7220357656478882), ('Inveravante_Inversiones_SL', 0.6925067901611328), ('Spains', 0.6856307983398438), ('Madrid', 0.6743447780609131), ('Spaniards', 0.6629219651222229)]


Es posible también buscar analogías. Por ejemplo, para resolver la analogía *man is to king, as woman is to ...*, se debe ejecutar la siguiente instrucción.

In [ ]:
wv.most_similar_cosmul(positive=['king','woman'],negative=['man'])

[('queen', 0.9314123392105103),
 ('monarch', 0.858533501625061),
 ('princess', 0.8476566076278687),
 ('Queen_Consort', 0.8150269985198975),
 ('queens', 0.8099815249443054),
 ('crown_prince', 0.808997631072998),
 ('royal_palace', 0.8027306795120239),
 ('monarchy', 0.801961362361908),
 ('prince', 0.800979733467102),
 ('empress', 0.7958388328552246)]

**Ejercicio**: Da respuesta a las siguientes analogías. 

*Eat is to ate, as go is to ...*

In [ ]:
wv.most_similar_cosmul(positive=['ate','go'],negative=['eat'])

[('went', 0.9165000319480896),
 ('gone', 0.8448726534843445),
 ('came', 0.816146969795227),
 ('ran', 0.8127626776695251),
 ('stayed', 0.7917163372039795),
 ('goes', 0.7796573042869568),
 ('sneaked', 0.774756669998169),
 ('snuck', 0.768629252910614),
 ('got', 0.7652180790901184),
 ('walked', 0.7639439702033997)]

*Madrid is to Spain, as Berlin is to ...*

In [ ]:
wv.most_similar_cosmul(positive=['Spain','Berlin'],negative=['Madrid'])

[('Germany', 0.9708642959594727),
 ('Austria', 0.8568971753120422),
 ('German', 0.8524766564369202),
 ('Hungary', 0.8441289663314819),
 ('Poland', 0.8382856845855713),
 ('Annita_Kirsten', 0.8365224599838257),
 ('Thielert_AG_Hamburg', 0.8243475556373596),
 ('Buffalo_Sabres_Jochen_Hecht', 0.8205474019050598),
 ('symbol_RSTI', 0.8203197121620178),
 ('Saxony', 0.8148468732833862)]

También es posible encontrar palabras extrañas dentro de un grupo de palabras. 

Por ejemplo, ¿cuál de las siguientes palabras no encaja en la lista ``[Jupyter, Earth, Saturday, Mars, Moon]``?

In [ ]:
wv.doesnt_match(['Jupyter','Earth','Saturday','Mars','Moon'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'Saturday'

**Ejercicio** ¿Qué palabra no encaja en la siguiente lista ``[April, May, September, Monday, July]``?

In [ ]:
wv.doesnt_match(['April', 'May', 'September', 'Monday', 'July'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'Monday'

## Glove y Fasttext

Además de word2vec, han aparecido otros embeddings. Los más conocidos son [Glove](https://nlp.stanford.edu/projects/glove/) y [FastText](https://fasttext.cc/). 

Como hemos explicado anteriormente, el método word2vec aprende la representación de las palabras mediante una red neuronal. En cambio estos otros dos embeddings funcionan de manera un poco distinta.

Glove se basa en técnicas de factorización de matrices. Para ello comienza construyendo una gran matriz con tantas filas y columnas como palabras. En esta matriz la entrada i,j indica el número de veces que la palabra i aparece en la misma frase que la palabra j. Seguidamente dicha matriz de co-ocurrencias se factoriza para producir una representación de baja dimensión.  

Tanto Glove como wor2vec tienen el problema de que no sirven para codificar palabras "raras" o que no aparecen en el vocabulario. Para resolver dicho problema surgió FastText. 

FastText es una extensión del modelo word2vec. En lugar de aprender vectores para cada palabra directamente, FastText representa cada palabra como un n-grama de caracteres. Por ejemplo, si tomamos la palabra artificial y usamos n=3, la representación de dicha palabra viene dada por <ar, art, rti, tif, ifi, fic, ici, ial, al> donde < y > indican respectivamente el principio y final de una palabra. 

Este método ayuda a capturar el signficado de palabras más cortas y permite comprender los sufijos y prefijos. Una vez que las palabras son partidas en n-gramas se entrena un modelo similar al de word2vec. Una ventaja de FastText es que funciona con palabras raras que no habían sido vistas anteriormente (cosa que no ocurría con los otros modelos). 



Desde el punto de vista de su uso no hay diferencias con respecto a word2vec. Vamos a comenzar descargando un modelo glove y otro modelo fasttext. 

In [ ]:
import gensim.downloader as api
wvGlove = api.load('glove-twitter-25')
wvFastText = api.load('fasttext-wiki-news-subwords-300')

**Ejercicio** ¿Qué otros modelos de fasttext y glove proporciona la librería gensym? Consulta lo que hace la función ``api.info()``.

In [ ]:
api.info()

{'corpora': {'20-newsgroups': {'checksum': 'c92fd4f6640a86d5ba89eaad818a9891',
   'description': 'The notorious collection of approximately 20,000 newsgroup posts, partitioned (nearly) evenly across 20 different newsgroups.',
   'fields': {'data': '',
    'id': 'original id inferred from folder name',
    'set': "marker of original split (possible values 'train' and 'test')",
    'topic': 'name of topic (20 variant of possible values)'},
   'file_name': '20-newsgroups.gz',
   'file_size': 14483581,
   'license': 'not found',
   'num_records': 18846,
   'parts': 1,
   'read_more': ['http://qwone.com/~jason/20Newsgroups/'],
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/20-newsgroups/__init__.py',
   'record_format': 'dict'},
  '__testing_matrix-synopsis': {'checksum': '1767ac93a089b43899d54944b07d9dc5',
   'description': '[THIS IS ONLY FOR TESTING] Synopsis of the movie matrix.',
   'file_name': '__testing_matrix-synopsis.gz',
   'parts': 1,
   're

**Ejercicio** Compara los resultados proporcionados por cada uno de los embeddings para los ejercicios presentados en el apartado anterior. Añade tantas celdas como necesites. 

Iré alternando los embeddings para que se pueda ver los resultados de una forma más visual

In [ ]:
try:
    vec_cameroon = wvGlove['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

In [ ]:
try:
    vec_cameroon = wvFastText['cameroon']
except KeyError:
    print("The word 'cameroon' does not appear in this model")

--------------------------------------------------------------------------------

In [ ]:
print(wvGlove.most_similar(positive=['car'], topn=5))

[('front', 0.936506986618042), ('on', 0.9070020318031311), ('table', 0.8939012885093689), ('truck', 0.8898833394050598), ('place', 0.8800071477890015)]


In [ ]:
print(wvFastText.most_similar(positive=['car'], topn=5)) # al ejecutar esto sube exponencialmente el uso de la CPU

[('cars', 0.7954095005989075), ('vehicle', 0.7870852947235107), ('non-car', 0.7824047803878784), ('automobile', 0.7790266275405884), ('super-car', 0.7759445905685425)]




---



In [ ]:
wvGlove.most_similar_cosmul(positive=['king','woman'],negative=['man'])

[('meets', 1.0724927186965942),
 ('crow', 1.03579580783844),
 ('hedgehog', 1.0280965566635132),
 ('prince', 1.024889349937439),
 ('hunter', 1.0226764678955078),
 ('mercy', 1.0204172134399414),
 ('queen', 1.0198343992233276),
 ('shepherd', 1.0195919275283813),
 ('soldier', 1.019392728805542),
 ('widow', 1.0162571668624878)]

In [ ]:
wvFastText.most_similar_cosmul(positive=['king','woman'],negative=['man'])

[('queen', 0.9390855431556702),
 ('queen-mother', 0.9078598618507385),
 ('king-', 0.8828967213630676),
 ('queen-consort', 0.882541835308075),
 ('child-king', 0.8680858016014099),
 ('monarch', 0.8670082688331604),
 ('ex-queen', 0.8654636740684509),
 ('princess', 0.8628991842269897),
 ('queen-', 0.8613532781600952),
 ('boy-king', 0.8604660630226135)]



---



In [ ]:
wvGlove.most_similar_cosmul(positive=['ate','go'],negative=['eat'])

[('gabi', 1.0315500497817993),
 ('isa', 1.0036530494689941),
 ('na', 0.9959880113601685),
 ('k', 0.995937168598175),
 ('po', 0.9776562452316284),
 ('agt', 0.9753057360649109),
 ('ba', 0.974004864692688),
 ('tou', 0.9657711982727051),
 ('dae', 0.9655082821846008),
 ('julia', 0.9648900628089905)]

In [ ]:
wvFastText.most_similar_cosmul(positive=['ate','go'],negative=['eat'])

[('went', 0.9635580778121948),
 ('gone', 0.8864149451255798),
 ('came', 0.8734760880470276),
 ('stayed', 0.8679504990577698),
 ('goes', 0.8597913980484009),
 ('followed', 0.8560923337936401),
 ('looked', 0.854744553565979),
 ('walked', 0.8542968034744263),
 ('got', 0.8524806499481201),
 ('wended', 0.852008581161499)]



---



In [ ]:
wvGlove.most_similar_cosmul(positive=['spain','berlin'],negative=['madrid'])

[('loоk', 1.1995182037353516),
 ('lооk', 1.165311336517334),
 ('латинский', 1.139757752418518),
 ('brussels', 1.1365525722503662),
 ('lоok', 1.1363283395767212),
 ('толковый', 1.120051383972168),
 ('герл', 1.115322470664978),
 ('트친분들에게', 1.114218831062317),
 ('cornwall', 1.1134241819381714),
 ('аt', 1.1115261316299438)]

In [ ]:
wvFastText.most_similar_cosmul(positive=['Spain','Berlin'],negative=['Madrid'])

[('Germany', 0.932011604309082),
 ('Germanies', 0.8789596557617188),
 ('Prussia', 0.8766506314277649),
 ('Poland', 0.8737272620201111),
 ('Austria', 0.8685898780822754),
 ('Britain', 0.8673689961433411),
 ('Europe', 0.8630521297454834),
 ('West-Berlin', 0.862476110458374),
 ('Denmark', 0.8581658601760864),
 ('France', 0.8580948710441589)]



---



In [ ]:
wvGlove.doesnt_match(['jupyter','earth','saturday','mars','moon'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'saturday'

In [ ]:
wvFastText.doesnt_match(['Jupyter','Earth','Saturday','Mars','Moon'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'Saturday'



---



In [ ]:
wvGlove.doesnt_match(['april', 'may', 'september', 'monday', 'july'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'may'

In [ ]:
wvFastText.doesnt_match(['April', 'May', 'September', 'Monday', 'July'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'Monday'



---



## Otros idiomas

Los embeddings son particulares de cada idioma por lo que la representación de palabras en inglés no nos sirve cuando estamos trabajando en español. Afortunadamente, existen [modelos preentrenados para nuestro idioma](https://github.com/dccuchile/spanish-word-embeddings).

Su uso es relativamente sencillo. Debemos descargar el fichero de los pesos y cargarlo. Seguidamente podremos usarlo como hemos visto anteriormente. Por ejemplo vamos a cargar un modelo FastText. Comenzamos descargando el modelo.

In [20]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec -O wiki.es.vec

--2022-05-25 13:51:22--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.es.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2594302560 (2.4G) [binary/octet-stream]
Saving to: ‘wiki.es.vec’

wiki.es.vec         100%[===================>]   2.42G  40.0MB/s    in 62s     

2022-05-25 13:52:25 (40.0 MB/s) - ‘wiki.es.vec’ saved [2594302560/2594302560]



A continuación mostramos las primeras filas del modelo, como se puede ver el documento que acabamos de descargar contiene en cada línea un n-grama (recordar qué es lo que hace FastText) y a continuación su representación en forma de vector. 

In [21]:
!head wiki.es.vec

985667 300
de -0.13075 -0.087659 -0.11427 -0.020641 0.11753 0.19687 0.054257 -0.0028717 0.062278 -0.10023 -0.050123 -0.026275 -0.057605 -0.13072 0.10147 0.15849 0.095493 0.051555 0.015874 -0.046374 0.098467 0.034867 0.039933 -0.1208 0.065478 -0.0098815 -0.13914 -0.043732 -0.015622 0.05665 -0.01476 -0.0054753 -0.047127 -0.21595 -0.015154 -0.0034798 0.058253 0.036444 -0.25157 0.060459 0.23842 0.017983 0.10673 -0.15889 0.23043 -0.078636 0.075394 -0.18431 -0.31417 0.084773 -0.14912 0.036904 -0.1144 0.025056 0.058607 0.059822 -0.17929 0.028468 0.16728 -0.020946 0.019714 0.0083937 0.032227 0.013204 0.06393 -0.19616 -0.043487 0.10124 -0.032762 0.17206 -0.062339 -0.10172 -0.31708 0.079012 -0.1232 -0.15504 -0.084187 -0.099777 0.16626 0.086791 0.001035 0.10478 0.12913 -0.0026416 0.061668 0.10004 -0.073838 0.167 0.10342 -0.05263 0.20125 0.23046 0.043589 0.19497 -0.0093385 -0.042631 -0.17599 -0.15208 0.23261 -0.10049 0.096678 -0.030501 0.060627 -0.27119 -0.11177 0.26739 0.205 -0.13012 0.051966 0.1

La carga del modelo es un poco distinta a lo visto anteriormente. En concreto debemos usar el siguiente código (notar que no cargamos todo el documento sino solo los 100000 primeros n-gramas). 

In [22]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'wiki.es.vec'
cantidad = 100000
wvFastTextSpanish = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

Ahora podemos utilizar las mismas funciones vistas anteriormente. 

**Ejercicio:** Encuentra las 5 palabras más similares a bicicleta. 

In [23]:
print(wvFastTextSpanish.most_similar(positive=['bicicleta'], topn=5))

[('bicicletas', 0.8180868029594421), ('bici', 0.7310570478439331), ('motocicleta', 0.6779677867889404), ('bike', 0.6354870796203613), ('moto', 0.5987207889556885)]


**Ejercicio:** Responde a la siguiente analogía. *Hombre es a actor como mujer es a...*

In [24]:
wvFastTextSpanish.most_similar_cosmul(positive=['actor','mujer'],negative=['hombre'])

[('actriz', 1.0298713445663452),
 ('bailarina', 0.9005751013755798),
 ('actrices', 0.900486409664154),
 ('dramaturga', 0.8913683891296387),
 ('coreógrafa', 0.884644627571106),
 ('locutora', 0.8787722587585449),
 ('presentadora', 0.8759288787841797),
 ('compositora', 0.8649977445602417),
 ('actoral', 0.8637272119522095),
 ('comediante', 0.8625256419181824)]

**Ejercicio:** Responde a la siguiente analogía. *Canta es a cantar como juega es a ...*

In [25]:
wvFastTextSpanish.most_similar_cosmul(positive=['cantar','juega'],negative=['canta'])

[('jugar', 0.9471045136451721),
 ('jugaba', 0.87227863073349),
 ('centrocampista', 0.8656566739082336),
 ('jugando', 0.8536456227302551),
 ('mediocampista', 0.8485172986984253),
 ('mediapunta', 0.8384414315223694),
 ('jugó', 0.837324857711792),
 ('jugarse', 0.8344488739967346),
 ('juegue', 0.8287681937217712),
 ('jugara', 0.827880859375)]

**Ejercicio:** Responde a la siguiente analogía. *Madrid es a España como Lisboa es a ...*

In [26]:
wvFastTextSpanish.most_similar_cosmul(positive=['españa','lisboa'],negative=['madrid'])

[('portugal', 0.9711438417434692),
 ('algarve', 0.872070848941803),
 ('estêvão', 0.8418089747428894),
 ('portuguesas', 0.8351491689682007),
 ('portuguesa', 0.8348497152328491),
 ('alentejo', 0.8336402177810669),
 ('portimão', 0.8250821232795715),
 ('portugues', 0.8246365189552307),
 ('portugueses', 0.8224292993545532),
 ('brasil', 0.8218298554420471)]

**Ejercicio:** Encuentra la palabra que no encaja en la lista ``[lunes, martes, septiembre, jueves, viernes]``

In [27]:
wvFastTextSpanish.doesnt_match(['lunes', 'martes', 'septiembre', 'jueves', 'viernes'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'septiembre'

**Ejercicio:** Descarga alguno de los otros modelos proporcionados en el [zoo de modelos](https://github.com/dccuchile/spanish-word-embeddings) y encuentra un ejemplo de analogía y otro de palabra que no encaje usando dicho modelo (añade tantas celdas como necesites).

In [12]:
!wget https://zenodo.org/record/3234051/files/embeddings-l-model.vec?download=1 -O unnotated.es.vec

--2022-05-25 13:42:11--  https://zenodo.org/record/3234051/files/embeddings-l-model.vec?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3446609027 (3.2G) [application/octet-stream]
Saving to: ‘unnotated.es.vec’

unnotated.es.vec    100%[===================>]   3.21G  19.1MB/s    in 5m 56s  

2022-05-25 13:48:08 (9.24 MB/s) - ‘unnotated.es.vec’ saved [3446609027/3446609027]



In [13]:
!head unnotated.es.vec

1313423 300
</s> -0.16791 -0.22665 0.078156 -0.18347 0.21301 0.055032 0.067502 0.13706 -0.14041 0.19155 -0.13608 0.040696 -0.036988 -0.099929 0.059987 -0.006037 0.07959 -0.086929 0.0032061 0.04053 -0.092542 0.086853 0.11532 0.15845 0.078315 -0.17212 -0.1129 -0.14736 -0.031068 0.10991 -0.21799 -0.064893 -0.061097 -0.015511 -0.09624 0.062328 -0.013588 -0.005091 -0.12687 0.088444 -0.17059 -0.010566 0.13244 -0.23103 -0.0057486 -0.031596 -0.011091 -0.015354 -0.022998 0.13553 0.25073 -0.0090087 0.048081 -0.022228 -0.050962 -0.064517 -0.11814 0.27844 -0.021293 -0.18498 0.15431 0.14207 0.053311 -0.091351 -0.17444 -0.028598 -0.067873 0.054908 -0.018664 -0.13587 0.011815 -0.13598 -0.10257 0.29452 -0.038179 -0.18362 0.045844 0.075999 -0.074984 -0.10918 -0.00094654 0.04368 -0.01443 0.019339 -0.093375 -0.11623 0.013624 -0.19358 0.052397 0.044992 0.058078 -0.033469 0.1245 -0.13038 -0.046593 0.032381 0.047781 -0.016469 0.004409 -0.037789 -0.13712 0.16659 -0.062807 0.28074 -0.04625 0.0022809 0.016488 

In [14]:
from gensim.models.keyedvectors import KeyedVectors
wordvectors_file_vec = 'unnotated.es.vec'
cantidad = 100000
wvFastTextSpanish2 = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [15]:
print(wvFastTextSpanish2.most_similar(positive=['bicicleta'], topn=5))

[('bici', 0.8596844673156738), ('motocicleta', 0.7343628406524658), ('moto', 0.7222307920455933), ('bicicletas', 0.7199786901473999), ('bicis', 0.6754755973815918)]


In [16]:
wvFastTextSpanish2.most_similar_cosmul(positive=['actor','mujer'],negative=['hombre'])

[('actriz', 1.029092788696289),
 ('dramaturga', 0.9112269878387451),
 ('coreógrafa', 0.9106628894805908),
 ('locutora', 0.88897305727005),
 ('compositora', 0.8822876214981079),
 ('escritora', 0.8775303959846497),
 ('actrices', 0.8739449977874756),
 ('pintora', 0.8698908090591431),
 ('presentadora', 0.868553876876831),
 ('actoral', 0.8673722743988037)]

In [17]:
wvFastTextSpanish2.most_similar_cosmul(positive=['cantar','juega'],negative=['canta'])

[('jugar', 0.9413612484931946),
 ('jugaba', 0.8551070690155029),
 ('jugando', 0.8297711610794067),
 ('juegan', 0.8239086866378784),
 ('jugaran', 0.8147255778312683),
 ('jugara', 0.8060213923454285),
 ('fútbol', 0.8019292950630188),
 ('jugado', 0.7989648580551147),
 ('jugaría', 0.7974690198898315),
 ('mediocampista', 0.7960951328277588)]

In [18]:
wvFastTextSpanish2.most_similar_cosmul(positive=['españa','lisboa'],negative=['madrid'])

[('portugal', 0.9645054340362549),
 ('brasil', 0.8456214666366577),
 ('grecia', 0.8347657322883606),
 ('algarve', 0.8329377770423889),
 ('portuguesa', 0.8286518454551697),
 ('francia', 0.8207144141197205),
 ('polonia', 0.8112043738365173),
 ('italia', 0.8110285401344299),
 ('portuguesas', 0.8109676241874695),
 ('finlandia', 0.8105317950248718)]

In [19]:
wvFastTextSpanish2.doesnt_match(['lunes', 'martes', 'septiembre', 'jueves', 'viernes'])

/usr/local/lib/python3.7/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'septiembre'

Recuerda guardar tu notebook en GitHub con la opción *Save in GitHub* del menú *File*.